### Setup

In [1]:
#download the image data
%cd /content/
!gdown https://drive.google.com/uc?id=1HpFpsQ7VojeJbXj8i9i8IeIzXrzQ5zOs
!unzip "nerf_data.zip"; rm "nerf_data.zip"
# !gdown https://drive.google.com/uc?id=1ahp0Ac3I_7Fm1usqPT_Rvgww5KaLvo6Y
# !unzip davis.zip; rm davis.zip

/content
Downloading...
From: https://drive.google.com/uc?id=1HpFpsQ7VojeJbXj8i9i8IeIzXrzQ5zOs
To: /content/nerf_data.zip
113MB [00:00, 170MB/s]
Archive:  nerf_data.zip
   creating: nerf_data/
   creating: nerf_data/kid-running/
   creating: nerf_data/kid-running/dense/
   creating: nerf_data/kid-running/dense/images/
  inflating: nerf_data/kid-running/dense/images/00069.png  
  inflating: nerf_data/kid-running/dense/images/00065.png  
  inflating: nerf_data/kid-running/dense/images/00044.png  
  inflating: nerf_data/kid-running/dense/images/00062.png  
  inflating: nerf_data/kid-running/dense/images/00030.png  
  inflating: nerf_data/kid-running/dense/images/00002.png  
  inflating: nerf_data/kid-running/dense/images/00013.png  
  inflating: nerf_data/kid-running/dense/images/00036.png  
  inflating: nerf_data/kid-running/dense/images/00034.png  
  inflating: nerf_data/kid-running/dense/images/00009.png  
  inflating: nerf_data/kid-running/dense/images/00056.png  
  inflating: nerf_da

In [2]:
%cd /content
!git clone https://github.com/rohaldb/Neural-Scene-Flow-Fields
%cd /content/Neural-Scene-Flow-Fields
!pip install -r requirements.txt

/content
Cloning into 'Neural-Scene-Flow-Fields'...
remote: Enumerating objects: 615, done.
remote: Counting objects: 100% (322/322), done.
remote: Compressing objects: 100% (255/255), done.
remote: Total 615 (delta 197), reused 143 (delta 67), pack-reused 293
Receiving objects: 100% (615/615), 99.68 MiB | 32.06 MiB/s, done.
Resolving deltas: 100% (319/319), done.
/content/Neural-Scene-Flow-Fields
     |████████████████████████████████| 37.2MB 137kB/s 


In [3]:
%cd /content
!git clone https://github.com/rohaldb/multi-mono-sf.git
%cd multi-mono-sf
!chmod -R +x ./
!pip install -r requirements.txt

/content
Cloning into 'multi-mono-sf'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 130 (delta 47), reused 105 (delta 29), pack-reused 0
Receiving objects: 100% (130/130), 55.91 MiB | 32.34 MiB/s, done.
Resolving deltas: 100% (47/47), done.
/content/multi-mono-sf
     |████████████████████████████████| 61kB 5.0MB/s 
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Execution
First run the standard preprocessing pipeline of the neural-scene-flow paper. This will:
1. Convert the intrinsics/extrinsics to a special readable format
2. Resize the images and run a single view depth model to produce disparity estimates
3. Produce optical flow estimates
4. Use the optical flow estimates to obtain motion masks

We do not use the depth map or the optical flow for custom experiments. 

In [5]:
%cd /content/Neural-Scene-Flow-Fields/nsff_scripts
# create camera intrinsics/extrinsic format for NSFF, same as original NeRF where it uses imgs2poses.py script from the LLFF code: https://github.com/Fyusion/LLFF/blob/master/imgs2poses.py
!python save_poses_nerf.py --data_path "/content/nerf_data/kid-running/dense/"
# Resize input images and run single view model
!python run_midas.py --data_path "/content/nerf_data/kid-running/dense/" --input_w 640 --input_h 360 --resize_height 288
# Run optical flow model (for easy setup and Pytorch version consistency, we use RAFT as backbond optical flow model, but should be easy to change to other models such as PWC-Net or FlowNet2.0)
!./download_models.sh
!python run_flows_video.py --model models/raft-things.pth --data_path /content/nerf_data/kid-running/dense/ --epi_threshold 1.0 --input_flow_w 768 --input_semantic_w 1024 --input_semantic_h 576

/content/Neural-Scene-Flow-Fields/nsff_scripts
Cameras 5
Images # 75
00000.png
00001.png
00002.png
00003.png
00004.png
00005.png
00006.png
00007.png
00008.png
00009.png
00010.png
00011.png
00012.png
00013.png
00014.png
00015.png
00016.png
00017.png
00018.png
00019.png
00020.png
00021.png
00022.png
00023.png
00024.png
00025.png
00026.png
00027.png
00028.png
00029.png
00030.png
00031.png
00032.png
00033.png
00034.png
00035.png
00036.png
00037.png
00038.png
00039.png
00040.png
00041.png
00042.png
00043.png
00044.png
00045.png
00046.png
00047.png
00048.png
00049.png
00050.png
00051.png
00052.png
00053.png
00054.png
00055.png
00056.png
00057.png
00058.png
00059.png
00060.png
00061.png
00062.png
00063.png
00064.png
00065.png
00066.png
00067.png
00068.png
00069.png
00070.png
00071.png
00072.png
00073.png
00074.png
(75, 17)
Done with imgs2poses
initialize
Minifying [288, 512] /content/nerf_data/kid-running/dense/
png
(714, 1270, 3) [288, 512]
(714, 1270, 3) [288, 512]
(714, 1270, 3) [288, 512]

Next we run the mono-sf code, which will produce:
1. Disparity maps
2. Scene flow estimates
3. Induced optical flow estimates

By default, it will not produce visualisations to acompany the stored tensors. To include them in the output, add `--save_vis=True` to `eval_kid_running.sh`

In [6]:
# Run mono-sf to obtain scene flow, induced optical flow and disparity estimates
%cd /content/multi-mono-sf/scripts
!./eval_kid_running.sh

/content/multi-mono-sf/scripts
2021-07-06 02:17:45 ==> Commandline Arguments
2021-07-06 02:17:45   batch_size: 1
2021-07-06 02:17:45   batch_size_val: 1
2021-07-06 02:17:45   calculate_disparity_scale: False
2021-07-06 02:17:45   checkpoint: checkpoints/checkpoint_kitti_selfsup.ckpt
2021-07-06 02:17:45   checkpoint_exclude_params: ['']
2021-07-06 02:17:45   checkpoint_include_params: ['*']
2021-07-06 02:17:45   checkpoint_mode: resume_from_latest
2021-07-06 02:17:45   conv_padding_mode: zeros
2021-07-06 02:17:45   correlation_cuda_enabled: False
2021-07-06 02:17:45   cuda: True
2021-07-06 02:17:45   evaluation: True
2021-07-06 02:17:45   finetuning: False
2021-07-06 02:17:45   lr_scheduler: None
2021-07-06 02:17:45   model: MonoSceneFlow_Multi
2021-07-06 02:17:45   num_workers: 16
2021-07-06 02:17:45   optimizer: Adam
2021-07-06 02:17:45   optimizer_amsgrad: False
2021-07-06 02:17:45   optimizer_betas: (0.9, 0.999)
2021-07-06 02:17:45   optimizer_eps: 1e-08
2021-07-06 02:17:45   optimi

Next, remove the unneeded content (nsf scene flow & nsf disparity), zip the content and store in google drive.

In [7]:
# Remove irrelevant folders
!rm -rf /content/nerf_data/kid-running/dense/flow_i1 #nsf optical flow
!rm -rf /content/nerf_data/kid-running/dense/disp #nsf disparity

In [ ]:
#save to gdrive
%cd /content/
!zip -r custom_nerf_data.zip ./nerf_data
!cp -r custom_nerf_data.zip /content/gdrive/MyDrive/Thesis/Data
!ls /content/gdrive/MyDrive/Thesis/Data